# Stage 03: Albert & DeBerta (Part 2)

## Imports & Load Dataset

In [1]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)
from pathlib import Path

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
import optuna
from optuna.pruners import MedianPruner

from src.training.metrics import compute_metrics, compute_metrics_from_logits
from src.training.tokenization_utils import make_tokenized_datasets
from src.training.loss import WeightedBCETrainer

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
    set_seed,
)

SEED = 42
set_seed(SEED)

ROOT = Path('.').resolve().parents[0]  # run from project root
TRAIN_PATH = ROOT / "data" / "processed" / "pcl_task1_train.csv"
DEV_PATH   = ROOT / "data" / "processed" / "pcl_task1_dev.csv"

OUTPUT_DIR = ROOT / "runs" / "optuna_task1"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

STUDY_DB = str(OUTPUT_DIR / "optuna_pcl_task1.db")  # sqlite db file
STUDY_NAME = "pcl_task1_binary"

train_df = pd.read_csv(TRAIN_PATH)
dev_df   = pd.read_csv(DEV_PATH)

# Keep only what we need
keep_cols = ["par_id", "text", "label_bin"]
train_df = train_df[keep_cols].copy()
dev_df   = dev_df[keep_cols].copy()

train_df["label_bin"] = train_df["label_bin"].astype(int)
dev_df["label_bin"]   = dev_df["label_bin"].astype(int)

print(train_df.shape, dev_df.shape)
print(train_df["label_bin"].value_counts())

/home/joshua_killa/.pyenv/versions/pcl-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(8375, 3) (2094, 3)
label_bin
0    7581
1     794
Name: count, dtype: int64


## Validation split

In [2]:
train_split, val_split = train_test_split(
    train_df,
    test_size=0.15,
    random_state=SEED,
    stratify=train_df["label_bin"], # Keep balance of classes
)

print("train:", train_split["label_bin"].value_counts().to_dict())
print("val  :", val_split["label_bin"].value_counts().to_dict())

ds_train_raw = Dataset.from_pandas(train_split.reset_index(drop=True))
ds_val_raw   = Dataset.from_pandas(val_split.reset_index(drop=True))
ds_dev_raw   = Dataset.from_pandas(dev_df.reset_index(drop=True))

train: {0: 6443, 1: 675}
val  : {0: 1138, 1: 119}


## Define search space for configurations 

In [3]:
# Fixed parameters
fixed_maxlen = 128  # 75% fit under 67 tokens    # trial.suggest_categorical("max_length", [96, 128, 192, 256])
fixed_batch_size = 16
fixed_epochs = 12

In [4]:
from transformers import TrainingArguments
print(TrainingArguments.__module__)
print(TrainingArguments)

transformers.training_args
<class 'transformers.training_args.TrainingArguments'>


In [5]:
def objective(trial: optuna.Trial):
    model_name = trial.suggest_categorical(
        "model_name",
        [
            "microsoft/deberta-v3-base",
            "albert-large-v2",
        ],
    )

    lr          = trial.suggest_float("lr", 5e-6, 5e-5, log=True)
    batch_size  = fixed_batch_size
    weight_decay= trial.suggest_float("weight_decay", 0.0, 0.1)
    warmup_ratio= trial.suggest_float("warmup_ratio", 0.0, 0.15)
    max_length  = fixed_maxlen   
    epochs      = fixed_epochs
    grad_accum  = trial.suggest_categorical("grad_accum", [1, 2]) # do [2, 4] if 16 doesnt fit
    pos_scale   = trial.suggest_float("pos_weight_scale", 0.75, 1.5)
    
    tok, ds_train, ds_val, _ = make_tokenized_datasets(model_name, max_length, ds_train_raw, ds_val_raw, ds_dev_raw)

    # base pos_weight = neg/pos (multiplier required to balance the classes GD update significance)
    y = np.array(train_split["label_bin"].values, dtype=int)
    n_pos = (y == 1).sum()
    n_neg = (y == 0).sum()
    base_pos_weight = (n_neg / max(n_pos, 1))
    pos_weight = torch.tensor(base_pos_weight * pos_scale, dtype=torch.float)

    cfg = AutoConfig.from_pretrained(model_name)
    cfg.num_labels = 1  # single logit
    model = AutoModelForSequenceClassification.from_pretrained(model_name, config=cfg)

    trial_dir = OUTPUT_DIR / f"trial_{trial.number:04d}"
    trial_dir.mkdir(parents=True, exist_ok=True)

    args = TrainingArguments(
        output_dir=str(trial_dir),
        seed=SEED,
        data_seed=SEED,

        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=grad_accum,

        num_train_epochs=epochs,
        weight_decay=weight_decay,
        warmup_ratio=warmup_ratio,

        eval_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",

        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,

        save_total_limit=1,

        fp16=torch.cuda.is_available(),   # safe; ignored on MPS/CPU
        report_to="none",
    )

    trainer = WeightedBCETrainer(
        model=model,
        args=args,
        train_dataset=ds_train,
        eval_dataset=ds_val,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.002)],
        pos_weight=pos_weight,
    )

    trainer.train()

    metrics = trainer.evaluate(ds_val)
    print(metrics)
    # Optuna optimizes this:
    return metrics["eval_f1"]

In [6]:
storage_url = f"sqlite:///{STUDY_DB}"

study = optuna.create_study(
    study_name=STUDY_NAME,
    direction="maximize",
    storage=storage_url,
    load_if_exists=True,
    pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=4),
)

print("Existing trials:", len(study.trials))

N_TRIALS = 50

study.optimize(objective, n_trials=N_TRIALS, gc_after_trial=True)

print("Best f1:", study.best_value)
print("Best params:", study.best_params)

[I 2026-02-20 05:57:17,884] Using an existing study with name 'pcl_task1_binary' instead of creating a new one.


Existing trials: 23


Loading weights: 100%|██████████| 25/25 [00:00<00:00, 504.40it/s, Materializing param=albert.pooler.weight]                                                             
AlbertForSequenceClassification LOAD REPORT from: albert-large-v2
Key                          | Status     | 
-----------------------------+------------+-
predictions.LayerNorm.weight | UNEXPECTED | 
predictions.dense.weight     | UNEXPECTED | 
predictions.decoder.bias     | UNEXPECTED | 
predictions.bias             | UNEXPECTED | 
predictions.LayerNorm.bias   | UNEXPECTED | 
predictions.dense.bias       | UNEXPECTED | 
classifier.bias              | MISSING    | 
classifier.weight            | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.
warmup_ratio is deprecated and will be removed in v5.2. Use `warmup

Epoch,Training Loss,Validation Loss


[W 2026-02-20 06:00:03,510] Trial 23 failed with parameters: {'model_name': 'albert-large-v2', 'lr': 3.5837564132203376e-05, 'weight_decay': 0.020947452766949804, 'warmup_ratio': 0.10103562321043406, 'grad_accum': 1, 'pos_weight_scale': 1.3035121475469165} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/joshua_killa/.pyenv/versions/pcl-env/lib/python3.10/site-packages/optuna/study/_optimize.py", line 206, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_98969/1002945683.py", line 73, in objective
    trainer.train()
  File "/home/joshua_killa/.pyenv/versions/pcl-env/lib/python3.10/site-packages/transformers/trainer.py", line 1412, in train
    return inner_training_loop(
  File "/home/joshua_killa/.pyenv/versions/pcl-env/lib/python3.10/site-packages/transformers/trainer.py", line 1742, in _inner_training_loop
    tr_loss_step = self.training_step(model, inputs, num_items_in_batch)
  File "/home/joshua_killa/.py

KeyboardInterrupt: 

## Train best configuration

In [ ]:
best = study.best_params
best_model = best["model_name"]

# rebuild datasets: train on (train+val), evaluate on dev
full_train = pd.concat([train_split, val_split], ignore_index=True)
ds_full_train_raw = Dataset.from_pandas(full_train.reset_index(drop=True))

tok = AutoTokenizer.from_pretrained(best_model, use_fast=True)

def tok_fn(batch):
    texts = [str(x) if x is not None else "" for x in batch["text"]]
    tokenized = tok(
        texts,
        truncation=True,
        padding="max_length",
        max_length=fixed_maxlen,
    )
    if "label_bin" in batch:
        tokenized["labels"] = [float(x) for x in batch["label_bin"]]
    return dict(tokenized)

# Tokenize datasets & truncate/pad them to max length
ds_full_train = ds_full_train_raw.map(tok_fn, batched=True)
ds_dev = ds_dev_raw.map(tok_fn, batched=True)

# Only rename if needed
if "labels" not in ds_full_train.column_names and "label_bin" in ds_full_train.column_names:
    ds_full_train = ds_full_train.rename_column("label_bin", "labels")
if "labels" not in ds_dev.column_names and "label_bin" in ds_dev.column_names:
    ds_dev = ds_dev.rename_column("label_bin", "labels")

# Remove unnecessary columns
ds_full_train = ds_full_train.remove_columns([c for c in ds_full_train.column_names if c not in ["input_ids", "attention_mask", "labels"]])
ds_dev = ds_dev.remove_columns([c for c in ds_dev.column_names if c not in ["input_ids", "attention_mask", "labels"]])

y_full = np.array(full_train["label_bin"].values, dtype=int)
n_pos = (y_full == 1).sum()
n_neg = (y_full == 0).sum()
base_pos_weight = (n_neg / max(n_pos, 1))
pos_weight = torch.tensor(base_pos_weight * best["pos_weight_scale"], dtype=torch.float)

cfg = AutoConfig.from_pretrained(best_model)
cfg.num_labels = 1
model = AutoModelForSequenceClassification.from_pretrained(best_model, config=cfg)

final_dir = OUTPUT_DIR / "best_final_model"
final_dir.mkdir(parents=True, exist_ok=True)

args = TrainingArguments(
    output_dir=str(final_dir),
    seed=SEED,
    data_seed=SEED,
    learning_rate=best["lr"],
    per_device_train_batch_size=fixed_batch_size,
    per_device_eval_batch_size=fixed_batch_size,
    gradient_accumulation_steps=best["grad_accum"],
    num_train_epochs=fixed_epochs,
    weight_decay=best["weight_decay"],
    warmup_ratio=best["warmup_ratio"],
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_total_limit=1,
    fp16=torch.cuda.is_available(),
    report_to="none",
)

trainer = WeightedBCETrainer(
    model=model,
    args=args,
    train_dataset=ds_full_train,
    eval_dataset=ds_dev,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)],
    pos_weight=pos_weight,
)

trainer.train()
dev_metrics = trainer.evaluate(ds_dev)
dev_metrics